# Implementing Callbacks in TensorFlow using the MNIST Dataset

이 코스에서는 패션 MNIST를 사용하여 분류를 수행하는 방법을 배웠습니다. 이 데이터 세트에는 의류 항목이 포함되어 있습니다. MNIST라는 또 다른 유사한 데이터 세트도 있으며, 여기에는 0에서 9까지의 숫자가 손으로 쓴 것이 포함되어 있습니다.

99% 정확도로 훈련되고 이 기준에 도달하면 훈련을 중단하는 MNIST 분류기를 작성하세요. 강의에서는 손실에 대해 이 작업을 수행하는 방법을 배웠지만 여기서는 정확도를 사용합니다.

일부 참고 사항:

네트워크는 9회 미만의 에포크에서 성공해야 합니다.
99% 이상에 도달하면 문자열 "Reached 99% accuracy so cancelling training!"을 출력하고 훈련을 중지해야 합니다.
추가 변수를 추가하는 경우 해당 변수의 이름이 수업에서 사용된 변수 이름과 동일하게 사용되도록 해야 합니다. 이것은 콜백의 함수 시그니처(매개변수 및 이름)에 중요합니다.

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

## Load and inspect the data

먼저 데이터를 로드하세요. 주의해야 할 몇 가지 사항이 있습니다.

mnist.npz 파일은 현재 작업 공간의 data 디렉토리 아래에 이미 포함되어 있습니다. 기본적으로 Keras의 load_data는 ~/.keras/datasets를 기준으로 상대적인 경로를 허용하지만 이 경우 다른 위치에 저장되어 있으므로 전체 경로를 지정해야 합니다.

load_data는 튜플 (x_train, y_train), (x_test, y_test) 형식으로 훈련 및 테스트 세트를 반환하지만 이 연습에서는 훈련 세트만 필요하므로 두 번째 튜플은 무시할 수 있습니다.

In [3]:
# Load the data

# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "data/mnist.npz")

# Discard test set
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data(path=data_path)
        
# Normalize pixel values
x_train = x_train / 255.0

In [4]:
data_shape = x_train.shape

print(f"There are {data_shape[0]} examples with shape ({data_shape[1]}, {data_shape[2]})")

There are 60000 examples with shape (28, 28)


## Defining your callback

In [5]:
class myCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:                 
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

## Create and train your model

이제 콜백을 정의했으므로 아래의 train_mnist 함수를 완성하세요.

이 과제를 통과하려면 모델을 10 에포크 동안 훈련하고 콜백이 9번째 에포크 전에 발생해야 합니다.

In [6]:
def train_mnist(x_train, y_train):

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([         
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ]) 
    
    model.compile(optimizer='adam',                   
                  loss='sparse_categorical_crossentropy',                   
                  metrics=['accuracy'])     
    
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

    return history

'train_mnist'를 호출하여 적절한 매개변수를 전달하여 훈련 이력을 얻으세요:

In [7]:
hist = train_mnist(x_train, y_train)

Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.1990 - accuracy: 0.9416
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0805 - accuracy: 0.9750: 0s - loss: 0.0824 - 
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0503 - accuracy: 0.9844
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0374 - accuracy: 0.9881
Epoch 5/10
1861/1875 [============================>.] - ETA: 0s - loss: 0.0277 - accuracy: 0.9908
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0276 - accuracy: 0.9909
